In [1]:
import gdown

file_id = '1l7Tsvp_1w0ZO3j6ktxlN-i_ZrR_MfcNW'
url = f'https://drive.google.com/uc?id={file_id}'
output1 = 'Customers.csv'
gdown.download(url, output1, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1l7Tsvp_1w0ZO3j6ktxlN-i_ZrR_MfcNW
To: c:\Users\yipin\OneDrive - National University of Singapore\Y3S1\DSA3101\Customer behaviour analysis\Customers.csv
100%|██████████| 6.87M/6.87M [00:00<00:00, 10.8MB/s]


'Customers.csv'

In [14]:
file_id = '1hX2-KzS0nwyEuhi9dWy22PzRUczXVpn7'
url = f'https://drive.google.com/uc?id={file_id}'
output2 = 'Orders Table.csv'
gdown.download(url, output2, quiet=False)

file_id = '1ATwj5A6yDHafhI7Az1DKgwX1UsDkuuAn'
url = f'https://drive.google.com/uc?id={file_id}'
output3 = 'Products Table.csv'
gdown.download(url, output3, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1hX2-KzS0nwyEuhi9dWy22PzRUczXVpn7
To: c:\Users\yipin\OneDrive - National University of Singapore\Y3S1\DSA3101\Customer behaviour analysis\Orders Table.csv
100%|██████████| 18.8M/18.8M [00:03<00:00, 5.67MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ATwj5A6yDHafhI7Az1DKgwX1UsDkuuAn
To: c:\Users\yipin\OneDrive - National University of Singapore\Y3S1\DSA3101\Customer behaviour analysis\Products Table.csv
100%|██████████| 2.08M/2.08M [00:00<00:00, 4.92MB/s]


'Products Table.csv'

In [15]:
import pandas as pd
import numpy as np

customers_df = pd.read_csv('Customers.csv')
customers_df.drop(columns=['Unnamed: 0'], inplace=True)

orders_df = pd.read_csv("Orders Table.csv")
orders_df.drop(columns=['Unnamed: 0'], inplace=True)
orders_df['Order Date'] = pd.to_datetime(orders_df['Order Date'], errors='coerce')

products_df = pd.read_csv("Products Table.csv")

In [17]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168946 entries, 0 to 168945
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   OrderID                      168946 non-null  int64         
 1   ProductID                    168946 non-null  int64         
 2   CustomerID                   168946 non-null  int64         
 3   Order Date                   102384 non-null  datetime64[ns]
 4   Shipping Date                168946 non-null  object        
 5   Actual days for shipping     168946 non-null  int64         
 6   Scheduled days for shipping  168946 non-null  int64         
 7   Late Delivery Risk           168946 non-null  int64         
 8   Discount Percentage          168946 non-null  float64       
 9   Returns                      168946 non-null  int64         
 10  Promotional Period           168946 non-null  int64         
 11  Order Region              

# Feature engineering
These are the features that we are going to feature engineering

It is important to refer to this additional data dictionary because the name can be abit misleading since I can't make the feature name too long

1. ordered: 0 means customer has not made any orders before, 1 means yes but only on 1 day, 2 means yes and on >1 day
2. total_order: Total number of orders customer has ever made
3. ave_monthly_orders: Average number of orders per month between the earliest and latest order of that customer, for customers who has made orders on >1 days, 0 if otherwise
4. days_last_order: The number of days between the latest date the customer made an order and the last date of the dataset. NA if never made any orders
5. only_promo_order: 1 means the customer has only place order/s the promotional period, 0 if otherwise.
6. total spend: Total spending a customer has ever made 
7. ave_monthly_spending: Average spending per month between the earliest and latest order, for customers who has made orders on >1 days, 0 if otherwise
8. most_ordered_cat: The category of products that the customer ordered the most
9. moc_ratio: (number of orders a customer made for their most ordered category) / (total orders the customer made)

In [19]:
# Create a copy of customers_df
customers_copy = customers_df.copy()

# Calculate the required new columns for each customer in customers_copy based on orders_df

# 1. ordered: 0 means customer has not made any orders before, 1 means yes but only on 1 day, 2 means yes and on >1 day
orders_per_day = orders_df.groupby('CustomerID')['Order Date'].nunique()
customers_copy['ordered'] = customers_copy['CustomerID'].map(lambda x: 2 if orders_per_day.get(x, 0) > 1 else (1 if orders_per_day.get(x, 0) == 1 else 0))

# 2. total_order: Total number of orders customer has ever made
total_orders = orders_df.groupby('CustomerID').size()
customers_copy['total_order'] = customers_copy['CustomerID'].map(total_orders).fillna(0).astype(int)

# 3. ave_monthly_orders: Average number of orders per month between the earliest and latest order of that customer
earliest_order = orders_df.groupby('CustomerID')['Order Date'].min()
latest_order = orders_df.groupby('CustomerID')['Order Date'].max()
tenure_months = (latest_order - earliest_order).dt.days / 30.44
ave_monthly_orders = total_orders / tenure_months
ave_monthly_orders[tenure_months < 1] = 0  # Setting 0 for customers with orders on <=1 day
customers_copy['ave_monthly_orders'] = customers_copy['CustomerID'].map(ave_monthly_orders).fillna(0)

# 4. days_last_order: The number of days between the latest date the customer made an order and the last date of the dataset
last_order_date = orders_df['Order Date'].max()
days_last_order = (last_order_date - latest_order).dt.days
customers_copy['days_last_order'] = customers_copy['CustomerID'].map(days_last_order)

# 5. only_promo_order: 1 means the customer has only placed order/s during the promotional period, 0 if otherwise
only_promo = orders_df.groupby('CustomerID')['Promotional Period'].all().astype(int)
customers_copy['only_promo_order'] = customers_copy['CustomerID'].map(only_promo).fillna(0).astype(int)

# Merging orders_df with products_df to get product prices and categories for each order
orders_with_products = orders_df.merge(products_df[['ProductID', 'Category', 'Price']], on='ProductID', how='left')

In [20]:
orders_with_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168946 entries, 0 to 168945
Data columns (total 17 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   OrderID                      168946 non-null  int64         
 1   ProductID                    168946 non-null  int64         
 2   CustomerID                   168946 non-null  int64         
 3   Order Date                   102384 non-null  datetime64[ns]
 4   Shipping Date                168946 non-null  object        
 5   Actual days for shipping     168946 non-null  int64         
 6   Scheduled days for shipping  168946 non-null  int64         
 7   Late Delivery Risk           168946 non-null  int64         
 8   Discount Percentage          168946 non-null  float64       
 9   Returns                      168946 non-null  int64         
 10  Promotional Period           168946 non-null  int64         
 11  Order Region              

In [21]:
# Calculating the required new columns for each customer in customers_copy based on the merged orders_with_products

# 6. total spend: Total spending a customer has ever made
total_spend = orders_with_products.groupby('CustomerID')['Price'].sum()
customers_copy['total_spend'] = customers_copy['CustomerID'].map(total_spend).fillna(0)

# 7. ave_monthly_spending: Average spending per month between the earliest and latest order
# For customers who have ordered on >1 days
ave_monthly_spending = total_spend / tenure_months
ave_monthly_spending[tenure_months < 1] = 0  # Setting to 0 for customers with orders on <=1 day
customers_copy['ave_monthly_spending'] = customers_copy['CustomerID'].map(ave_monthly_spending).fillna(0)

# 8. most_ordered_cat: The category of products that the customer ordered the most
most_ordered_cat = orders_with_products.groupby('CustomerID')['Category'].agg(lambda x: x.mode()[0] if not x.mode().empty else '')
customers_copy['most_ordered_cat'] = customers_copy['CustomerID'].map(most_ordered_cat)

In [22]:
# 9
most_ordered_counts = orders_with_products.groupby(['CustomerID', 'Category']).size().unstack(fill_value=0)

# Update customers_copy to handle cases with no orders for moc_ratio calculation
def calculate_moc_ratio(row):
    if row['total_order'] > 0 and row['most_ordered_cat']:
        return most_ordered_counts.loc[row['CustomerID'], row['most_ordered_cat']] / row['total_order']
    else:
        return 0

# Apply function to calculate `moc_ratio`
customers_copy['moc_ratio'] = customers_copy.apply(calculate_moc_ratio, axis=1)


In [4]:
customers_copy.head()

,CustomerID,Age,Gender,Payment Method,Churn,Tenure,Complain,NumberOfDeviceRegistered,SatisfactionScore,CouponUsed,...,HourSpendOnApp,ordered,total_order,ave_monthly_orders,days_last_order,only_promo_order,total_spend,ave_monthly_spending,most_ordered_cat,moc_ratio
0,0,33,Female,Cash,0,0.0,0,4,5,1.0,...,4.0,0,0,0.000000,NaN,0,0.00,0.000000,NaN,0.0
1,1,17,Female,Cash,1,8.0,1,3,2,1.0,...,2.0,2,2,0.285822,575.0,0,64.75,9.253474,Clothing,1.0
2,2,42,Female,Cash,0,9.0,0,5,1,2.0,...,4.0,2,4,0.143924,256.0,0,105.58,3.798883,Clothing,0.5
3,3,37,Female,Cash,1,0.0,1,5,5,1.0,...,4.0,2,2,0.189657,755.0,0,31.18,2.956758,Clothing,1.0
4,4,63,Female,Cash,0,11.0,0,4,4,1.0,...,2.0,0,0,0.000000,NaN,0,0.00,0.000000,NaN,0.0


In [23]:
df=customers_copy.copy()
df.drop(columns=['Age', 'Gender', 'Churn', 'Complain'], inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CustomerID                100000 non-null  int64  
 1   Payment Method            100000 non-null  object 
 2   Tenure                    100000 non-null  float64
 3   NumberOfDeviceRegistered  100000 non-null  int64  
 4   SatisfactionScore         100000 non-null  int64  
 5   CouponUsed                100000 non-null  float64
 6   CashbackAmount            100000 non-null  float64
 7   HourSpendOnApp            100000 non-null  float64
 8   ordered                   100000 non-null  int64  
 9   total_order               100000 non-null  int32  
 10  ave_monthly_orders        100000 non-null  float64
 11  days_last_order           81577 non-null   float64
 12  only_promo_order          100000 non-null  int32  
 13  total_spend               100000 non-null  fl

# Label each segment 

In our labelling, high means top 25% and low means 25%. The numbers are taken from the EDA.

1. Discount_seekers: "only_promo_order" = 1 or high "CouponUsed” ie >=2 or high “Cashback Amount” ie >=205.45
2. Loyal High-Spenders: High "total spend" ie >= 62.4 & high "ave_monthly_orders" ie >= 0.32 & long "tenure" ie >= 14
3. Occasional Shoppers: Low "ave_monthly_orders" ie <= 0.13 & low 'HourSpendOnApp' ie <=2
4. Tech-Savvy Users: High "HourSpendOnApp" ie >= 4 & high "NumberOfDeviceRegistered ie >=4
5. Single-Category Shoppers: high "moc_ratio" ie >=1, where moc stands for most ordered category
6. Long-Tenured Non-Buyers: "ordered" = 0 & high "Tenure" ie >=14

In [24]:
df['segment_1'] = (df['only_promo_order'] == 1) | (df['CouponUsed'] >= 2) | (df['CashbackAmount'] >= 205.45)
df['segment_2'] = (df['total_spend'] >= 62.4) & (df['ave_monthly_orders'] >= 0.32) & (df['Tenure'] >= 14)
df['segment_3'] = (df['ave_monthly_orders'] <= 0.13) & (df['HourSpendOnApp'] <= 2)
df['segment_4'] = (df['HourSpendOnApp'] >= 4) & (df['NumberOfDeviceRegistered'] >= 4)
df['segment_5'] = df['moc_ratio'] >= 1
df['segment_6'] = (df['ordered'] == 0) & (df['Tenure'] >= 14)

# Display a few rows to verify the results
df[['CustomerID', 'segment_1', 'segment_2', 'segment_3', 'segment_4', 'segment_5', 'segment_6']].head()

,CustomerID,segment_1,segment_2,segment_3,segment_4,segment_5,segment_6
0,0,False,False,False,True,False,False
1,1,True,False,True,False,True,False
2,2,True,False,False,True,True,False
3,3,False,False,False,True,True,False
4,4,False,False,True,False,False,False


In [25]:
# Define total number of customers and segment names
total_customers = 100000
segments = ['Discount_seekers', 'Loyal High-Spenders', 'Occasional Shoppers', 'Tech-Savvy Users', 'Single-Category Shoppers', 'Long-Tenured Non-Buyers']
segment_counts = {}

# Calculate the number of True values and percentages for each segment
for i in range(1, 7):  # Segment columns are named segment_1 to segment_6
    count_true = df[f'segment_{i}'].sum()
    percentage = (count_true / total_customers) * 100
    segment_counts[segments[i-1]] = (count_true, percentage)  # Mapping each segment name correctly

# Create the report based on the calculated counts and percentages
report = [f"{count_true} customers are {segment} ({percentage:.2f}%)"
          for segment, (count_true, percentage) in segment_counts.items()]

report


['60318 customers are Discount_seekers (60.32%)',
 '1029 customers are Loyal High-Spenders (1.03%)',
 '31543 customers are Occasional Shoppers (31.54%)',
 '27150 customers are Tech-Savvy Users (27.15%)',
 '72854 customers are Single-Category Shoppers (72.85%)',
 '8561 customers are Long-Tenured Non-Buyers (8.56%)']